In [ ]:
import joblib

model1 = joblib.load("/content/drive/MyDrive/Data centre/IDS_WORKSPACE/model/binary_model.pkl")
scaler1 = joblib.load("/content/drive/MyDrive/Data centre/IDS_WORKSPACE/model/scaler_binary.pkl")
imputer1 = joblib.load("/content/drive/MyDrive/Data centre/IDS_WORKSPACE/model/imputer_binary.pkl")

model2 = joblib.load("/content/drive/MyDrive/Data centre/IDS_WORKSPACE/model/attack_type_model.pkl")
scaler2 = joblib.load("/content/drive/MyDrive/Data centre/IDS_WORKSPACE/model/scaler_attack.pkl")
imputer2 = joblib.load("/content/drive/MyDrive/Data centre/IDS_WORKSPACE/model/imputer_attack.pkl")
le_attack = joblib.load("/content/drive/MyDrive/Data centre/IDS_WORKSPACE/model/attack_label_encoder.pkl")
feature_names = joblib.load("/content/drive/MyDrive/Data centre/IDS_WORKSPACE/model/features_names.pkl")


In [ ]:
import numpy as np
import pandas as pd

class IDSPipeline:
    def __init__(self, model1, model2, imputer1, scaler1, imputer2, scaler2, le_attack, feature_names):
        self.model1 = model1
        self.model2 = model2
        self.imputer1 = imputer1
        self.scaler1 = scaler1
        self.imputer2 = imputer2
        self.scaler2 = scaler2
        self.le_attack = le_attack
        self.feature_names = feature_names

    def preprocess(self, features, imputer, scaler):
        # Convert input to 2D array
        features = np.array(features).reshape(1, -1)
        # Replace inf/nan
        features = np.nan_to_num(features, posinf=np.nan, neginf=np.nan)
        # Impute missing
        features = imputer.transform(features)
        # Scale
        features = scaler.transform(features)
        # Back to DataFrame (keep feature order)
        return pd.DataFrame(features, columns=self.feature_names)

    def predict(self, features):
        # Binary classification
        X_bin = self.preprocess(features, self.imputer1, self.scaler1)
        binary_pred = self.model1.predict(X_bin)[0]

        if binary_pred == 0:
            return "BENIGN"
        else:
            # Attack classification
            X_attack = self.preprocess(features, self.imputer2, self.scaler2)
            attack_pred = self.model2.predict(X_attack)[0]
            return self.le_attack.inverse_transform([attack_pred])[0]

In [ ]:
import joblib

pipeline_dict = {
    "model1": pipeline.model1,
    "scaler1": pipeline.scaler1,
    "imputer1": pipeline.imputer1,
    "model2": pipeline.model2,
    "scaler2": pipeline.scaler2,
    "imputer2": pipeline.imputer2,
    "le_attack": pipeline.le_attack,
    "feature_names": pipeline.feature_names
}

joblib.dump(pipeline_dict, "/content/drive/MyDrive/Data centre/IDS_WORKSPACE/model/IDS_Pipeline_joblib.pkl")
print("Pipeline converted to joblib successfully!")


Pipeline converted to joblib successfully!
